# 서울 랜드마크 이미지 분류 경진대회

# 목차

0. 라이브러리 설치
1. 환경설정
2. 데이터 불러오기
3. 데이터 전처리
4. 모델 정의
5. 모델 학습
6. 모델 예측
7. 결과 제출

# 0. 라이브러리 설치

# 1. 환경설정

### 1.1 GPU 세팅 및 확인

In [1]:
import torch
import torch.nn as nn

# GPU가 인식되면 GPU 사용, 아니면 CPU 사용
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#GPU 체크 및 할당
if torch.cuda.is_available():
    print('Device:', device)
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

Device: cuda
There are 1 GPU(s) available.
We will use the GPU: NVIDIA A100-SXM4-80GB MIG 7g.80gb


## 1.2 하이퍼 파라미터 세팅

In [2]:
CFG = {
    'IMG_SIZE':128, #이미지의 가로세로 픽셀 사이즈
    'EPOCHS':50, #에포크 (학습할 때 반복 수)
    'LEARNING_RATE':2e-2, #학습률
    'BATCH_SIZE':12, #배치사이즈 (모델에 한번에 들어갈 데이터 양)
    'SEED':41, # 랜덤 시드
}

## 1.3 랜덤시드 고정

In [3]:
# Seed 고정
import random
import numpy as np
import os

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

# 2. 데이터 불러오기

## 2.1 CSV 데이터

In [4]:
import pandas as pd
label_df = pd.read_csv('train.csv')
label_df.head()

,file_name,label
0,001.PNG,9
1,002.PNG,4
2,003.PNG,1
3,004.PNG,1
4,005.PNG,6


In [5]:
label_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 723 entries, 0 to 722
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  723 non-null    object
 1   label      723 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.4+ KB


## 2.2. 이미지 데이터

In [11]:
from glob import glob

In [12]:
img_path_list = []

In [13]:
img_path_list.extend(glob(os.path.join('/train/*.PNG')))

In [14]:
len(label_df)

723

In [26]:
import os
from glob import glob

def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    img_path_list.extend(glob(os.path.join(data_dir, '*.PNG')))
    # 이미지 경로 가져온 후 정리 및 정렬
    img_path_list.sort()  # 파일 이름을 기준으로 정렬

    # 정답값 리스트 저장 (여기서 label_df는 적절한 데이터프레임이어야 합니다.)
    label_list.extend(label_df['label'])

    return img_path_list, label_list

def get_test_data(data_dir):
    img_path_list = []

    # 이미지 경로 가져온 후 정리 및 정렬
    img_path_list.extend(glob(os.path.join(data_dir, '*.PNG')))
    img_path_list.sort()  # 파일 이름을 기준으로 정렬

    return img_path_list


all_img_path, all_label = get_train_data('study/숙련/MLseoul/train')
test_img_path = get_test_data('study/숙련/MLseoul/test')


In [29]:
import os
from glob import glob

def get_train_data(data_dir):
    img_path_list = []
    label_list = []

    # 이미지 경로 가져온 후 정리 및 정렬
    img_path_list.extend(glob(os.path.join(data_dir, '*.PNG')))
    img_path_list.sort()  # 파일 이름을 기준으로 정렬

    # 정답값 리스트 저장
    label_list.extend(label_df['label'])

    return img_path_list, label_list

def get_test_data(data_dir):
    img_path_list = []

    # 이미지 경로 가져온 후 정리 및 정렬
    img_path_list.extend(glob(os.path.join(data_dir, '*.PNG')))
    img_path_list.sort()  # 파일 이름을 기준으로 정렬

    return img_path_list


all_img_path, all_label = get_train_data('study/숙련/MLseoul/train')
test_img_path = get_test_data('study/숙련/MLseoul/test')

In [30]:
all_img_path

[]

# 3. 데이터 전처리

In [ ]:
import torchvision.datasets as datasets # 이미지 데이터셋 집합체
import torchvision.transforms as transforms # 이미지 변환 툴

from torch.utils.data import DataLoader # 학습 및 배치로 모델에 넣어주기 위한 툴
from torch.utils.data import DataLoader, Dataset

import cv2

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None): #필요한 변수들을 선언
        self.transforms = transforms # 이미지 변환 여부
        self.train_mode = train_mode # 학습용인지 테스트용인지 여부
        self.img_path_list = img_path_list # 이미지 경로 리스트
        self.label_list = label_list # 정답값 리스트

    def __getitem__(self, index): #index번째 data를 return
        # index번째 이미지 가져오기
        img_path = self.img_path_list[index]
        image = cv2.imread(img_path)

        if self.transforms is not None: # 이미지 변환
            image = self.transforms(image)

        if self.train_mode: # 학습시에만 적용, 테스트할때는 미적용
            label = self.label_list[index]
            return image, label
        else:
            return image

    def __len__(self): #길이 return
        return len(self.img_path_list)

#### 3.1.1 데이터 확인

In [ ]:
import matplotlib.pyplot as plt

# 임의의 데이터셋을 만들어서 하나 출력해보기
tempdataset = CustomDataset(all_img_path, all_label, train_mode=False)
plt.imshow(tempdataset.__getitem__(4))

## 3.2 Train / Validation Split

In [ ]:
# 데이터 세트 분리
train_len = int(len(all_img_path)*0.75)
Vali_len = int(len(all_img_path)*0.25)

train_img_path = all_img_path[:train_len]
train_label = all_label[:train_len]

vali_img_path = all_img_path[train_len:]
vali_label = all_label[train_len:]

print('train set 개수 : ', train_len)
print('vaildation set 개수 : ', Vali_len)

## 3.3 transforms

### 과제1 : Albumentation을 사용해서 더 많은 증강도구를 빠르게 사용해보세요.

In [ ]:
!pip install albumentations
import albumentations as A


In [ ]:
# 증강을 포함하는 변환(transform) 정의
transform = A.Compose([
    A.Resize(width=256, height=256),  # 이미지 크기 조정
    A.HorizontalFlip(p=0.5),  # 50% 확률로 좌우 반전
    A.RandomBrightnessContrast(p=0.2),  # 20% 확률로 밝기/대비 조절
    A.Rotate(limit=30, p=0.5),  # 50% 확률로 최대 30도 회전
])


In [ ]:
from torchvision import transforms as T

train_transform = T.Compose([
    T.ToPILImage(),  # Numpy 배열에서 PIL 이미지로 변환
    T.Resize([CFG['IMG_SIZE'], CFG['IMG_SIZE']]),  # 이미지 사이즈 조정
    T.ToTensor(),  # 이미지 데이터를 텐서로 변환
    T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))  # 이미지 정규화
])

test_transform = T.Compose([
    T.ToPILImage(),
    T.Resize([CFG['IMG_SIZE'], CFG['IMG_SIZE']]),
    T.ToTensor(),
    T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])


In [ ]:
#원래꺼
train_transform = transforms.Compose([
                    transforms.ToPILImage(), #Numpy배열에서 PIL이미지로
                    transforms.Resize([CFG['IMG_SIZE'], CFG['IMG_SIZE']]), #이미지 사이즈 변형
                    transforms.ToTensor(), #이미지 데이터를 tensor
                    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)) #이미지 정규화
                    ])

test_transform = transforms.Compose([
                    transforms.ToPILImage(),
                    transforms.Resize([CFG['IMG_SIZE'], CFG['IMG_SIZE']]),
                    transforms.ToTensor(),
                    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                    ])

## 3.4 Dataloader

In [ ]:
#CustomDataset 클래스를 통하여 train dataset생성
train_dataset = CustomDataset(train_img_path, train_label, train_mode=True, transforms=train_transform)
#만든 train dataset를 batch_size에 맞게 데이터 나누기
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

#vaildation 에서도 적용
vali_dataset = CustomDataset(vali_img_path, vali_label, train_mode=True, transforms=test_transform)
vali_loader = DataLoader(vali_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

train_batches = len(train_loader)
vali_batches = len(vali_loader)

print('학습용 데이터 개수',train_len,'= 학습용 배치 개수', train_batches, 'X 12(배치 사이즈)')
print('검증용 데이터 개수',Vali_len, '= 검증용 배치 개수', vali_batches, 'X 12(배치 사이즈)')

# 4. 모델 정의

### 과제2 : timm에서 다양한 모델들을 사용해보세요

In [ ]:
!pip install timm

In [ ]:
import timm

model = timm.create_model('resnet18', pretrained=True, num_classes=10)
model = model.to('cuda')

In [ ]:
import timm

# ResNet 모델 생성
model = timm.create_model('resnet18', pretrained=True)

# 모델 출력
print(model)


In [ ]:
import pandas as pd
import os
from glob import glob
from PIL import Image
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset

# CSV 파일에서 레이블 데이터 로드
label_df = pd.read_csv('/content/drive/MyDrive/jbig/6주차/JBIG 서울 이미지 분류2주차/train.csv')

class CustomDataset(Dataset):
    def __init__(self, data_dir, label_df, transform=None):
        self.data_dir = data_dir
        self.label_df = label_df
        self.transform = transform
        self.img_paths = self._load_img_paths()

    def _load_img_paths(self):
        img_path_list = []
        img_path_list.extend(glob(os.path.join(self.data_dir, '*.PNG')))
        img_path_list.sort()  # 파일 이름을 기준으로 정렬
        return img_path_list

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        label = self.label_df.iloc[idx]['label']
        return img, label

# 데이터 전처리 및 변환 정의
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 학습 데이터셋 및 테스트 데이터셋 객체 생성
train_dataset = CustomDataset('/content/drive/MyDrive/jbig/6주차/JBIG 서울 이미지 분류2주차/train', label_df, transform=preprocess)
test_dataset = CustomDataset('/content/drive/MyDrive/jbig/6주차/JBIG 서울 이미지 분류2주차/test', label_df, transform=preprocess)





In [ ]:
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms

# 하이퍼파라미터 설정
NUM_CLASSES = 10  # 클래스 개수에 따라 적절히 설정
BATCH_SIZE = 32
LEARNING_RATE = 0.001
NUM_EPOCHS = 10


# 데이터셋 경로
train_img_path = "/content/drive/MyDrive/jbig/6주차/JBIG 서울 이미지 분류2주차/train"
train_label = "/content/drive/MyDrive/jbig/6주차/JBIG 서울 이미지 분류2주차/train.csv"

# 데이터 전처리 및 데이터로더 생성
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),  # ResNet은 보통 224x224 크기의 입력을 사용합니다.
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])
train_dataset = CustomDataset(train_img_path, train_label, train_mode=True, transforms=train_transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# ResNet 모델 생성
model = timm.create_model('resnet18', pretrained=True)
# 모델의 마지막 레이어를 새로운 fully connected 레이어로 대체하여 클래스 수에 맞게 설정
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)

# GPU를 사용할 수 있는 경우 GPU로 모델 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print("학습 완료!")


In [ ]:
#원본
from torchvision import models
from torchvision.models import efficientnet_b3 as efficientnet
from torchvision.models import resnet50 as resnet
import torch.nn as nn
from torch.nn import functional as F
from torch.nn import CrossEntropyLoss
import torch.optim as optim

model = models.efficientnet_b3(pretrained=False)
model

In [ ]:
#원본
# 1000 = 모델의 마지막 층의 out_features가 1000개이기 때문에 1000으로 설정
# 10 = 우리가 분류할 정답값이 10개이므로 10으로 설정
# 주의 : 모델을 바꾸면 해당 모델 마지막 층의 out_features에 따라 1000에서 값을 변경할 것!!
model.fc = nn.Linear(1000, 10)
model = model.to(device)
model

# 5. 모델 학습

### 과제 3 : 스케쥴러를 추가해보세요.

In [ ]:
#과제 3
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader

# 스케줄러를 위한 파라미터 설정
step_size = 5  # 스텝 사이즈
gamma = 0.1  # 학습률을 감소시킬 배율

# 데이터로더 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 손실 함수와 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 스케줄러 설정
scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

scheduler

### 과제 4 : 얼리스탑핑을 추가해보세요.

In [ ]:
#과제4
import numpy as np

def train_with_early_stopping(model, train_loader, criterion, optimizer, scheduler, num_epochs=10, patience=3):
    best_loss = np.inf
    current_patience = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
        scheduler.step()

        # 검증 손실 계산 (학습 중이므로 검증은 생략합니다)

        # 얼리 스탑핑 조건 확인
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            current_patience = 0
        else:
            current_patience += 1
            if current_patience >= patience:
                print("Early stopping triggered. Training stopped.")
                return


In [ ]:
#원본
from tqdm import tqdm

# 오류를 계산할 평가 방식 정의
criterion = torch.nn.CrossEntropyLoss()
# 학습을 어떻게 할지 방법론 정의
optimizer = torch.optim.SGD(params = model.parameters(), lr = CFG["LEARNING_RATE"])

def train(model, optimizer, train_loader, device):
    model.to(device)
    n = len(train_loader)
    best_acc = 0

    for epoch in range(1,CFG["EPOCHS"]+1): #에포크(반복수) 설정
        model.train() #모델 학습
        running_loss = 0.0

        for img, label in tqdm(iter(train_loader)):
            img, label = img.to(device), label.to(device) #배치 데이터
            optimizer.zero_grad() #전방향 학습, 기울기 초기화

            # Data -> Model -> Output
            output = model(img) # 입력 -> 출력
            loss = criterion(output, label) #오차 계산

            # 역전파
            loss.backward() #역전파 학습 (기울기 계산)
            optimizer.step() #기울기 업데이트
            running_loss += loss.item()

        print('[%d] Train loss: %.10f' %(epoch, running_loss / len(train_loader)))


        #Validation set 평가
        model.eval() #evaluation 과정에서 사용하지 않아야 하는 layer들을 알아서 off 시키도록 하는 함수
        vali_loss = 0.0
        correct = 0
        with torch.no_grad(): #파라미터 업데이트 안하기 때문에 no_grad 사용
            for img, label in tqdm(iter(vali_loader)):
                img, label = img.to(device), label.to(device)

                output = model(img)# 입력 -> 출력
                vali_loss += criterion(output, label)#오차 계산
                pred = output.argmax(dim=1, keepdim=True)  #11개의 class중 가장 값이 높은 것을 예측 label로 추출
                correct += pred.eq(label.view_as(pred)).sum().item() #예측값과 실제값이 맞으면 1 아니면 0으로 합산
        vali_acc = 100 * correct / len(vali_loader.dataset)
        print('Vail set: Loss: {:.4f}, Accuracy: {}/{} ( {:.0f}%)\n'.format(vali_loss / len(vali_loader), correct, len(vali_loader.dataset), 100 * correct / len(vali_loader.dataset)))

        # 이전에 가장 잘 학습했던 모델의 정확도보다 높을 경우 모델 저장
        if best_acc < vali_acc:
            best_acc = vali_acc
            torch.save(model.state_dict(), './best_model.pth') #이 디렉토리에 best_model.pth을 저장
            print('Model Saved.')

In [ ]:
train(model, optimizer, train_loader, device)

# 6. 모델 예측

In [ ]:
#원본
def predict(model, test_loader, device):
    model.eval()
    model_pred = []
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            img = img.to(device)

            pred_output = model(img)
            pred_output = pred_output.argmax(dim=1, keepdim=True).squeeze(1)

            model_pred.extend(pred_output.tolist())
    return model_pred

In [ ]:
#원본
test_dataset = CustomDataset(test_img_path, None, train_mode=False, transforms=test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

# Validation Accuracy가 가장 뛰어난 모델을 불러옵니다.
checkpoint = torch.load('/content/best_model.pth')

model = models.efficientnet_b3(pretrained=False)
model.fc = nn.Linear(1000, 10)
model = model.to(device)

model.load_state_dict(checkpoint)

# Inference
preds = predict(model, test_loader, device)
preds[0:5]

# 7. 결과 제출

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/jbig/6주차/JBIG 서울 이미지 분류2주차/sample_submission.csv')
submission['label'] = preds
submission.to_csv('/content/drive/MyDrive/jbig/6주차/JBIG 서울 이미지 분류2주차/sample_submission.csv', index=False)